<img width="8%" alt="Plotly.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Plotly.png" style="border-radius: 15%">

# Plotly - Follow reach by day

**Tags:** #plotly #html #csv #image #content #analytics #reach #views #likes #comments #shares #metric

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook tracks your daily reach over the past two weeks of publication on LinkedIn. In the event that we are unable to access your views (impressions), the total number of interactions (likes, comments, and shares) will be used as a metric.

## Input

### Import libraries

In [ ]:
import plotly.graph_objects as go
from naas_drivers import gsheet
import pandas as pd
import os
from datetime import date, datetime, timedelta
import naas_data_product

### Setup variables
**Inputs**
- `entity_dir`: This variable represents the entity directory.
- `entity_name`: This variable holds the entity name.
- `input_dir`: Input directory to retrieve file from.
- `input_file`: Name of the file to be retrieved.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `title`: Graph title.
- `linkedin_url`: Entity LinkedIn URL used as graph source.

**Outputs**
- `output_dir`: This variable is used for storing the path to the directory where the output files will be saved.

In [ ]:
# Inputs
entity_dir = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_dir")
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_name")
input_dir = os.path.join(entity_dir, "content-engine", date.today().isoformat())
input_file = "content"
spreadsheet_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "abi_spreadsheet")
sheet_name = "CONTENT"
title = "Views"
linkedin_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "linkedin_url")

# Outputs
output_dir = os.path.join(entity_dir, "content-engine", date.today().isoformat())

## Model

### Set outputs

In [ ]:
csv_output = os.path.join(output_dir, "content_trend.csv")
html_output = os.path.join(output_dir, "content_trend.html")
image_output = os.path.join(output_dir, "content_trend.png")

### Get content published

In [ ]:
# Get data from pickle or Google Sheets spreadsheet
df_input = pload(input_dir, input_file)    
if df_input is None:
    df_input = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)

# Filter on this week and last week + entity
if len(df_input) > 0:
    df_input = df_input[(df_input["ENTITY"] == entity_name) & df_input["SCENARIO"].isin([TW, LW])]

# Display result
print("Input data:", len(df_input))
df_input.head(1)

### Create trend dataframe

In [ ]:
DATE_FORMAT = "%Y-%m-%d"
PERIOD = "%Y-%m-%d"
PERIOD_TEXT = "This day"

def get_trend(
    df_init,
    col_date,
    col_value,
    agg_value,
    entity_name,
    title
):
    # Init variable
    df = df_init.copy()
    
    # Check if views != 0
    if col_value == "VIEWS" and col_value in df.columns and df[col_value].sum() == 0:
        df["INTERACTIONS"] = df["LIKES"] + df["COMMENTS"] + df["SHARES"]
        col_value = "INTERACTIONS"
        title = "Interactions"

    # Groupby period
    if agg_value == "sum":
        df[col_value] = df[col_value].astype(float)
    df[col_date] = pd.to_datetime(df[col_date].str[:-6]).dt.strftime(DATE_FORMAT)
    df = df.groupby(col_date, as_index=False).agg({col_value: agg_value})

    # Rename column
    to_rename = {col_date: "DATE", col_value: "VALUE"}
    df = df.rename(columns=to_rename)

    # Reindex value
    d = datetime.now().date()
    d2 = date.today() - timedelta(days=date.today().weekday() + 7)
    idx = pd.date_range(d2, d, freq="D")
    df.set_index("DATE", drop=True, inplace=True)
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    df["DATE"] = pd.DatetimeIndex(df.index)

    # Groupby month
    df["DATE"] = pd.to_datetime(df["DATE"], format=DATE_FORMAT).dt.strftime(PERIOD)
    df = df.groupby("DATE", as_index=False).agg({"VALUE": "sum"})

    # Calc variation
    df.loc[:, "VALUE_COMP"] = 0.
    df.loc[:, "VARV"] = 0.
    df.loc[:, "VARP"] = 1.
    for index, row in df.iterrows():
        if index > 0:
            n = df.loc[df.index[index], "VALUE"]
            n_1 = df.loc[df.index[index - 1], "VALUE"]
            df.loc[df.index[index], "VALUE_COMP"] = n_1
            df.loc[df.index[index], "VARV"] = n - n_1
            if n_1 > 0:
                df.loc[df.index[index], "VARP"] = (n - n_1) / abs(n_1)
    df = df.fillna(0.)

    # Plotly: Date display
    df["DATE_D"] = pd.to_datetime(df["DATE"], format=PERIOD).dt.strftime("%a %d %b")

    # Plotly: Value display
    df["VALUE_D"] = (
        "<b><span style='font-family: Arial;'>"
        + df["VALUE"].map("{:,.0f}".format).str.replace(",", " ")
        + "</span></b>"
    )

    # Plotly: Variation display
    df["VARV_D"] = df["VARV"].map("{:,.0f}".format).str.replace(",", " ")
    df.loc[df["VARV"] >= 0, "VARV_D"] = "+" + df["VARV_D"]
    df["VARP_D"] = df["VARP"].map("{:,.0%}".format).str.replace(",", " ")
    df.loc[df["VARP"] >= 0, "VARP_D"] = "+" + df["VARP_D"]

    # Plotly: hovertext
    df["TEXT"] = (
        "<b><span style='font-size: 14px;'>"
        + df["DATE_D"].astype(str)
        + ": "
        + df["VALUE_D"]
        + "</span></b><br>"
        "<span style='font-size: 12px;'>"
        + df["VARV_D"]
        + " ("
        + df["VARP_D"]
        + ")</span>"
    )
    
    # Add graph title
    df.insert(loc=0, column="ENTITY", value=entity_name)
    df.insert(loc=1, column="SCENARIO", value=pd.to_datetime(df["DATE"]).dt.strftime("W%W-%Y"))
    
    # Map colors
    df["COLOR"] = df["SCENARIO"].map(MAPPING_COLORS)
    return df.reset_index(drop=True), title

df_trend, title = get_trend(
    df_input,
    col_date="PUBLISHED_DATE",
    col_value="VIEWS",
    agg_value="sum",
    entity_name=entity_name,
    title=title
)
# print(f'Title: {title}')
# df_trend

### Create title and logo

In [ ]:
# Groupby weeks
df = df_trend.groupby(["SCENARIO"], as_index=False).agg({"VALUE": "sum"})
df["ORDER"] = df["SCENARIO"].str[-4:] + df["SCENARIO"].str[1:3]
df = df.sort_values(by="ORDER", ascending=True)

# Calc data
total_n1 = df.loc[df.index[0], "VALUE"]
total = df.loc[df.index[-1], "VALUE"]
varv = total - total_n1
varp = 0
if total_n1 != 0:
    varp = varv / total_n1

# Create value to displayed
total_d = "{:,.0f}".format(total).replace(",", " ")
varv_d = "{:,.0f}".format(varv).replace(",", " ")
varp_d = "{:,.0%}".format(varp).replace(",", " ")
if varv >= 0:
    varv_d = f"+{varv_d}"
    varp_d = f"+{varp_d}"
title_full = f"<b><span style='font-size: 20px;'>{title}</span></b><br><span style='font-size: 18px;'>{TW}: {total_d} | {varv_d} ({varp_d}) vs last week</span>"

# Logo
logo = None
if varv > 0:
    logo = arrow_up
elif varv > -0.2:
    logo = arrow_right
else:
    logo = arrow_down
print("Title:", title_full)

### Create vertical barchart

In [ ]:
def create_barchart(
    df,
    title=None,
    logo=None,
    linkedin_url=None,
    label="DATE_D",
    value="VALUE",
    value_d="VALUE_D",
    text="TEXT"
):
    # Init
    fig = go.Figure()

    # Create traces
    scenarios = df["SCENARIO"].unique()
    for s in scenarios:
        tmp_df = df[df["SCENARIO"] == s]
        # Create bar
        fig.add_trace(
            go.Bar(
                x=tmp_df[label],
                y=tmp_df[value],
                name=s,
                marker=dict(color=tmp_df["COLOR"]),
                text=tmp_df[value],
                textposition="outside",
                hoverinfo="text",
                hovertext=tmp_df["TEXT"],
                showlegend=True
            )
        )
        
    # Add logo
    fig.add_layout_image(
        dict(
            source=logo,
            xref="paper",
            yref="paper",
            x=0.01,
            y=1.06,
            sizex=0.12,
            sizey=0.12,
            xanchor="right",
            yanchor="bottom",
        )
    )
    # Add legend
    fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5
        )
    )
    
    # Add annotation
    fig.add_annotation(
        text=f"<i>Source: <a href='{linkedin_url}'>{linkedin_url}</a> / Created at: {date.today().isoformat()}</i>",
        font=dict(family="Arial", color="black"),
        x=0,
        y=-0.35,
        xref="paper",
        yref="paper",
        xanchor="left",
        yanchor="bottom",
        arrowcolor="white",
    )

    # Update layout
    fig.update_layout(
        title=title,
        title_x=0.09,
        title_font=dict(family="Arial", color="black"),
        paper_bgcolor="#ffffff",
        plot_bgcolor="#ffffff",
        width=1200,
        height=600,
        margin_pad=10,
        margin_b=120
    )
    fig.show()
    return fig

fig = create_barchart(df_trend, title_full, logo, linkedin_url)

## Output

### Save and share your csv file

In [ ]:
# Save your dataframe in CSV
df_trend.to_csv(csv_output, index=False)

### Save and share your graph in HTML


In [ ]:
# Save your graph in HTML
fig.write_html(html_output)

# Share output with naas
html_link = naas.asset.add(html_output, override_prod=True, params={"inline": True})

# -> Uncomment the line below to remove your asset
# naas.asset.delete(html_output)

### Save and share your graph in image


In [ ]:
# Save your graph in PNG
fig.write_image(image_output)

# Share output with naas
image_link = naas.asset.add(image_output, override_prod=True, params={"inline": True})

# -> Uncomment the line below to remove your asset
# naas.asset.delete(image_output)